# TensorBoard

![](https://www.tensorflow.org/images/mnist_tensorboard.png)

- [TensorBoard: Visualizing Learning  |  TensorFlow](https://www.tensorflow.org/get_started/summaries_and_tensorboard)
- [텐서보드 · 텐서플로우 문서 한글 번역본](https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/how_tos/summaries_and_tensorboard/)

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

/usr/local/anaconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Set the Log Directory

In [2]:
import time
from pathlib import Path

LOGPATH = Path('./logs')
timestamp_log = time.strftime('%y%m%d%H%M')

logdir = str(LOGPATH / f'lin-reg-{timestamp_log}')

## Read Datasets

In [3]:
DIM_INPUTS = 11
DIM_OUTPUTS = 1

In [4]:
dat = pd.read_csv('datasets/winequality-red.csv')
X_total, Y_total = dat.values[:, 0:-1], dat.values[:, [-1]]
X_train, X_test, Y_train, Y_test = train_test_split(X_total, Y_total, test_size=0.2)

## Model

In [5]:
with tf.name_scope('Inputs'):
    X = tf.placeholder(tf.float32, shape=[None, DIM_INPUTS], name='Input')

with tf.name_scope('Targets'):
    Y = tf.placeholder(tf.float32, shape=[None, DIM_OUTPUTS], name='Target')

with tf.name_scope('Weights'):
    W = tf.Variable(tf.random_normal([DIM_INPUTS, DIM_OUTPUTS]), name='Weights')

with tf.name_scope('Bias'):
    b = tf.Variable(tf.random_normal([DIM_OUTPUTS]), name='Bias')

## Optimization

In [6]:
with tf.name_scope('Hypothesis'):
    H = tf.matmul(X, W) + b

with tf.name_scope('Train'):
    with tf.name_scope('Cost'):
        cost = tf.reduce_mean(tf.square(H - Y))

        # Track cost as a scalar
        tf.summary.scalar('Cost', cost)

    with tf.name_scope('Optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-4)
        
    with tf.name_scope('Gradients'):
        gradient = tf.reduce_mean((H - Y) * X) * 2
        
        # Track gradient as a scalar
        tf.summary.scalar('Gradients/mean', gradient)

    train = optimizer.minimize(cost)

## Make a Session

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(logdir, sess.graph)
merged = tf.summary.merge_all()

## Train the Model

In [8]:
costs = []
for step in range(2001):
    _, summary = sess.run([train, merged], feed_dict={X: X_train, Y: Y_train})
    writer.add_summary(summary, step)
    writer.flush()

## Close the Session

In [9]:
writer.close()
sess.close()

## Outputs on TensorBoard


```bash
tensorboard --logdir=path/to/log-directory
```

![](./images/tensorboard-scalars.png)
![](./images/tensorboard-graphs.png)